# XOR Neural Network: Excel → NumPy → PyTorch
This notebook is aligned with **`xor_nn_workbook_no_macros.xlsx`**.

⚠️ Note: `openpyxl` does **not** evaluate Excel formulas. So formula cells (like MSE) may appear as `None` when loaded with `data_only=True`.
To avoid this, we:
- read parameters with `data_only=True` (numbers)
- read the **MSE formula text** with `data_only=False`
- compute MSE in NumPy from the forward pass (source of truth)


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from openpyxl import load_workbook

XLSX = Path("xor_nn_workbook_no_macros.xlsx")


## 1) XOR dataset

In [ ]:
X = np.array([[0,0],[0,1],[1,0],[1,1]], dtype=float)
y = np.array([[0],[1],[1],[0]], dtype=float)
pd.DataFrame(np.hstack([X,y]), columns=["x1","x2","y"])


## 2) Read weights from the Excel workbook
Workbook `02_Forward` parameter cells:
- W1: B5:C6
- b1: E6:F6
- W2: H6:H7
- b2: J6
- MSE formula cell: I15


In [ ]:
wb = load_workbook(XLSX, data_only=True)
ws = wb["02_Forward"]

W1 = np.array([[ws["B5"].value, ws["C5"].value],
               [ws["B6"].value, ws["C6"].value]], dtype=float)
b1 = np.array([ws["E6"].value, ws["F6"].value], dtype=float)
W2 = np.array([[ws["H6"].value],
               [ws["H7"].value]], dtype=float)
b2 = np.array([ws["J6"].value], dtype=float)

W1, b1, W2, b2


## 3) Forward pass in NumPy + MSE
We compute y_hat and MSE directly. This is the reliable comparison baseline.


In [ ]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

Z1 = X @ W1 + b1
A1 = sigmoid(Z1)
y_hat = A1 @ W2 + b2
mse_np = float(np.mean((y_hat - y)**2))

pd.DataFrame(np.hstack([X, y, y_hat]), columns=["x1","x2","y","y_hat"]), mse_np


## 4) Read the Excel MSE cell safely
Because `openpyxl` won't calculate formulas, we show:
- the **formula text** from Excel
- the cached computed value if it exists (often `None` unless Excel recalculated & saved)
- and we compare to our NumPy MSE.


In [ ]:
# Formula text (always available)
wb_f = load_workbook(XLSX, data_only=False)
ws_f = wb_f["02_Forward"]
mse_formula = ws_f["I15"].value

# Cached computed value (may be None)
wb_val = load_workbook(XLSX, data_only=True)
ws_val = wb_val["02_Forward"]
mse_cached = ws_val["I15"].value  # may be None

print("Excel MSE cell:", "02_Forward!I15")
print("Formula text   :", mse_formula)
print("Cached value   :", mse_cached)
print("NumPy MSE      :", mse_np)

# If cached is present, show absolute difference
if mse_cached is not None:
    print("abs(diff)      :", abs(float(mse_cached) - mse_np))


## 5) Visualize the network (Excel-style diagram)
Blue = positive weight, Red = negative weight, thickness = |weight|.


In [ ]:
def draw_nn(W1, W2, *,
            node_size=800,
            max_linewidth=4.0,
            min_linewidth=0.2,
            layer_labels=("Input", "Hidden", "Output"),
            title="Neural Network Diagram"):
    W1 = np.asarray(W1, dtype=float)
    W2 = np.asarray(W2, dtype=float)

    n_input, n_hidden = W1.shape
    n_hidden2, n_output = W2.shape
    if n_hidden2 != n_hidden:
        raise ValueError(f"Hidden size mismatch: W1 hidden={n_hidden}, W2 hidden={n_hidden2}")

    max_abs = max(np.max(np.abs(W1)), np.max(np.abs(W2)), 1e-9)

    x_input, x_hidden, x_output = 0.10, 0.50, 0.90

    def y_positions(n, top=0.80, bottom=0.20):
        if n == 1:
            return np.array([(top + bottom) / 2])
        return np.linspace(top, bottom, n)

    y_in  = y_positions(n_input,  top=0.75, bottom=0.25)
    y_hid = y_positions(n_hidden, top=0.75, bottom=0.25)
    y_out = y_positions(n_output, top=0.65, bottom=0.35)

    def lw(w):
        w = abs(w) / max_abs
        return min_linewidth + w * (max_linewidth - min_linewidth)

    fig, ax = plt.subplots(figsize=(7, 7))

    for i in range(n_input):
        for j in range(n_hidden):
            w = W1[i, j]
            ax.plot([x_input, x_hidden], [y_in[i], y_hid[j]],
                    linewidth=lw(w),
                    color=("blue" if w > 0 else "red"),
                    alpha=0.75)

    for i in range(n_hidden):
        for j in range(n_output):
            w = W2[i, j]
            ax.plot([x_hidden, x_output], [y_hid[i], y_out[j]],
                    linewidth=lw(w),
                    color=("blue" if w > 0 else "red"),
                    alpha=0.75)

    ax.scatter([x_input]*n_input,   y_in,  s=node_size, color="#cfe9ff", edgecolors="black", zorder=3)
    ax.scatter([x_hidden]*n_hidden, y_hid, s=node_size, color="#ccffcc", edgecolors="black", zorder=3)
    ax.scatter([x_output]*n_output, y_out, s=node_size, color="#ffd6d6", edgecolors="black", zorder=3)

    ax.text(x_input,  0.12, layer_labels[0], ha="center", fontsize=12)
    ax.text(x_hidden, 0.12, layer_labels[1], ha="center", fontsize=12)
    ax.text(x_output, 0.12, layer_labels[2], ha="center", fontsize=12)

    ax.set_xlim(0, 1); ax.set_ylim(0, 1)
    ax.axis("off")
    ax.set_title(title)
    plt.show()

draw_nn(W1, W2, title="XOR Network (weights from Excel v2 workbook)")


## 6) Train XOR in NumPy (full backprop)

In [ ]:
def dsigmoid(a):
    return a*(1-a)

def forward(X, W1, b1, W2, b2):
    Z1 = X @ W1 + b1
    A1 = sigmoid(Z1)
    y_hat = A1 @ W2 + b2
    return Z1, A1, y_hat

def train_numpy(X, y, hidden=4, lr=0.5, steps=5000, seed=0):
    rng = np.random.default_rng(seed)
    W1 = rng.normal(scale=0.5, size=(2, hidden))
    b1 = np.zeros((hidden,))
    W2 = rng.normal(scale=0.5, size=(hidden, 1))
    b2 = np.zeros((1,))
    losses = []

    for _ in range(steps):
        Z1, A1, y_hat = forward(X, W1, b1, W2, b2)
        err = (y_hat - y)
        losses.append(float(np.mean(err**2)))

        N = X.shape[0]
        dY = (2.0/N) * err
        gW2 = A1.T @ dY
        gb2 = np.sum(dY, axis=0)

        dA1 = dY @ W2.T
        dZ1 = dA1 * dsigmoid(A1)
        gW1 = X.T @ dZ1
        gb1 = np.sum(dZ1, axis=0)

        W1 -= lr * gW1
        b1 -= lr * gb1
        W2 -= lr * gW2
        b2 -= lr * gb2

    return (W1,b1,W2,b2), np.array(losses)

params, losses = train_numpy(X, y, hidden=4, lr=0.5, steps=5000)
losses[-1]


In [ ]:
plt.figure()
plt.plot(losses)
plt.xlabel("step")
plt.ylabel("MSE")
plt.title("NumPy training loss on XOR")
plt.show()

W1n,b1n,W2n,b2n = params
_, _, y_hat_n = forward(X, W1n,b1n,W2n,b2n)
pd.DataFrame(np.hstack([X,y,y_hat_n]), columns=["x1","x2","y","y_hat"])


## 7) Train XOR in PyTorch (autograd)

In [ ]:
pip install torch

In [ ]:
import torch
import torch.nn as nn

torch.manual_seed(0)

X_t = torch.tensor(X, dtype=torch.float32)
y_t = torch.tensor(y, dtype=torch.float32)

class XORNet(nn.Module):
    def __init__(self, hidden=4):
        super().__init__()
        self.fc1 = nn.Linear(2, hidden)
        self.act = nn.Sigmoid()
        self.fc2 = nn.Linear(hidden, 1)
    def forward(self, x):
        return self.fc2(self.act(self.fc1(x)))

model = XORNet(hidden=4)
opt = torch.optim.SGD(model.parameters(), lr=0.5)
loss_fn = nn.MSELoss()

losses_t = []
for step in range(5000):
    opt.zero_grad()
    y_pred = model(X_t)
    loss = loss_fn(y_pred, y_t)
    loss.backward()
    opt.step()
    losses_t.append(loss.item())

losses_t[-1]


In [ ]:
plt.figure()
plt.plot(losses_t)
plt.xlabel("step")
plt.ylabel("MSE")
plt.title("PyTorch training loss on XOR")
plt.show()

with torch.no_grad():
    y_pred = model(X_t).numpy()
pd.DataFrame(np.hstack([X,y,y_pred]), columns=["x1","x2","y","y_hat"])
